In [2]:
import numpy as np
import pandas as pd
import math

import matplotlib.pyplot as plt
from astropy.io import ascii

In [3]:
def F_nu(t_p,nu_p,F_nup,nu_sed):
    '''Calculates a Synchrotron Self-Absorbed Spectrum for given input parameters.
    
    Parameters:
    t_p (days): Time of SSA peak - should be single number
    nu_p (GHz): Frequency of SSA peak - should be single number
    F_nup (mJy): Flux of SSA peak - should be single number
    nu (GHz): Frequencies at which you want the SSA spectrum to be calculated at. Likely an array.
    
    Outputs:
    Fnu (mJy): the flux of the SSA. This is an array with values associated with each value of input array nu.
    '''
    
    m = 0.88 #using stAandard model
    p = 3.0 #using standard model
    a = (2*m)+0.5
    b = (p+5-(6*m))/2
    t=t_p
    Fnu = F_nup*1.582*(t/t_p)**a*(nu_sed/nu_p)**(5/2)*(1-np.exp(-(t/t_p)**(-(a+b))*(nu_sed/nu_p)**(-(p+4)/2)))
    
    return Fnu


def SSA_props(t_p,nu_p,F_nup,D,f=0.5,alpha=1,vw=100,epsilon_b=0.1):
    '''Calculates Synchroton Self-Absorption properties for given input parameters.
    
    Inputs:
    t_p (days): Time of SSA peak - likely a single number
    nu_p (GHz): Frequency of SSA peak - likely a single number
    F_nup (mJy): Flux of SSA peak - likely a single number
    D (Mpc): distance to SN - likely a single number
    f (unitless): filling factor (fraction of emitting region). default is 0.5
    alpha (unitless): ratio charged particles to magnetic field (epsilon_e/epsilon_b). default 1
    v_w (km/s): wind speed. default = 100
    epsilon_b (unitless): fraction of shock energy into B-fields. default = 0.1
    
    Outputs:
    R (cm): radius of material 
    B : magnetic field flux
    E : intermal energy of emitting material
    v (km/s): expansion velocity of material
    M (1d-5 solar masses per year): inferred mass loss rate of progenitor'''
    

    #Radius
    R = 4.0e14*(alpha)**(-1/19)*(f/0.5)**(-1/19)*(F_nup)**(9/19)*(D)**(18/19)*(nu_p/5)**(-1)

    # Magnetic field flux
    B = 1.1*(alpha)**(-4/19)*(f/0.5)**(-4/19)*(F_nup)**(-2/19)*(D)**(-4/19)*(nu_p/5)

    #Internal energy of the emitting material
    E = (1/epsilon_b)*((B**2)/(8* 3.142))*((4*3.142*f*R**3)/3)

    #expansion velocity in km/s
    v = (R/t_p)*1.1574e-10

    #pre-explosion mass-loss in 1e-5 solar mass per year
    M = 1.0*(alpha)**(-8/19)*epsilon_b*(f)**(-8/19)*(F_nup)**(-4/19)*(D)**(-4/19)*(nu_p/5)*(t_p/10)*(vw/1000)
    
   

    return R,B,E,v,M


def taufreefree(M,R,nu):
    '''Calculates the free free optical depth for a given set of parameters.
    
    Inputs:
    M (1d-5 solar masses per year): mass loss rate
    R (cm): radius of emitting material
    nu (GHz): Frequencies at which you want tau-ff to be calculated at. Likely an array.
    
    Outputs:
    tau_ff (unitless): the free-free optical depth, calculated at same frequences as input array nu'''

    Z_ave = 5.4  # Average metallicity 1= pure H. 5.4 for a massive star
    miu = 1.9 # mean molecular weight of electrons. 1= pure H  1.9 is for a massive star.
    vw_cgs = 100 * 1e5 # assumed wind velocity in cgs (cm/s). Take this as 1000 * 10^5 for now. (i.e. 1000 km/s in cgs)
    T = 10**4 # temperature of the material absorbing in K.  10^4 is a good starting point. 
        
    M_cgs =  M * 1e-5 * 6.307e+25 #mass loss rate in cgs units
    
    tau_ff = 2.021e25*M_cgs**2*Z_ave/(miu**2*nu**(2.1)*R**3*vw_cgs**2*T**(1.35))  
    
    return tau_ff

def den(M,R):
    vw_cgs = 100 * 1e5
    M_cgs =  M * 1e-5 * 6.307e+25 #mass loss rate in cgs units 
        #density of the CSM
    density = M_cgs/(4*3.142*R**2*vw_cgs)
    
    return density
#freqs,SED = F_nu(1224.288326 , 8,  0.0272)       
#R2,B2,E2,v2,M2 = SSA_props(1224.288326 , 8,  0.0272,880) 
#tauff=taufreefree(M2,R2,freqs)
#tauff=taufreefree(4,3e14,np.array([2,5,7]))
#print(freqs)
#print(tauff)
#print(R2,B2,E2,v2,M2)

## Make a Dense GRID

In [34]:
##MAKES DENSER GRIDS.

#Define a set of times post-explosion that I want to calculate this for; run a massive grid for each and write out.
tp = [100,200,500,800,1000,1200,1500,1800,2000,2200,2500,2800,3000,3200,4000,5000,6000,7000,9000]
#tp = [100,200,500,800,1000,1200,1500,1800,2000,2200,2500,2800,3000,3067,3200,3255,3500] #days post explosion
#tp = [4000,5000,6000,7000,9000]
#D = 1070.14 # distance to PS11aop
D = 543.4 #Distancce to PS11vo

F_p = np.logspace(np.log10(0.0001),np.log10(1.0),num=700) #mJy (this is an array evenly spaced in log between 0.01 annd 1)
nu_p = np.logspace(np.log10(0.00005),np.log10(100),num=1000) #GHz (this is an array evenly spaced in log between 0.05 annd 50)

### Define empty arrays of the values that you want to save for each value in the grid you are searching over: ###


for t in tp:
    file_out = 'SSAgrid_11vo_d'+np.str(t)+'.csv'
    #file_out = 'SSAgrid_11vo_d'+np.str(t)+'.csv'
    Fp_g = [] #peak flux
    nup_g =[] #peak frequency
    R_g =[] #radius
    B_g = [] #Bfield
    vsh_g = [] #velocity of shock
    M_g = [] #mass loss rate
    den_g = []
    for F in F_p:
        for nu in nu_p:
        
            #calculate Mass loss rate, Radius, velocity, etc.
            R,B,E,v,M = SSA_props(t,nu,F,D)
        
            # Append the values from this loop into the arrays that we defined above:
            Fp_g.append(F) 
            nup_g.append(nu) 
            R_g.append(R) 
            B_g.append(B) 
            vsh_g.append(v) 
            M_g.append(M)

    # Write out the results into a data file that you can use later:
    data = [Fp_g,nup_g,R_g,B_g,vsh_g,M_g]
    names = ['F_peak','nu_peak','Radius','Bfield','v_shock','Mdot']
    ascii.write(data,file_out,names=names,overwrite=True,format='csv')
    print('Finished tp=',t)

Finished tp= 100
Finished tp= 200
Finished tp= 500
Finished tp= 800
Finished tp= 1000
Finished tp= 1200
Finished tp= 1500
Finished tp= 1800
Finished tp= 2000
Finished tp= 2200
Finished tp= 2500
Finished tp= 2800
Finished tp= 3000
Finished tp= 3200
Finished tp= 4000
Finished tp= 5000
Finished tp= 6000
Finished tp= 7000
Finished tp= 9000


## Attempt to turn this into a Light Curve 

In [ ]:
#Pick a velocity and Mdot that we are interested in. 
#PS11aop
#velocity = 3500 #km/s
#Mdot = 1.14 #1e-5 Msun/year
#D = 1070.14 

#PS11vo:
velocity = 5000 #1537 # 10059
correction_factor = 0.1*0.5**(-8./19.)
Mdot2 = 10.0#0.348 #2.46 #0.348
Mdot = Mdot2*correction_factor
print(Mdot2,Mdot)
D = 543.4


#Fiducial:
#velocity = 5000 #km/s
#correction_factor = 0.1*0.5**(-8./19.)
#Mdot2 = 1
#Mdot = Mdot2 * correction_factor 
#print(Mdot)
#D = 1070.14 #Mpc

#fileout_LC = "BestFit_11aop_SSA_v"+np.str(velocity)+"_M"+np.str(Mdot)+"e-5.csv"
fileout_LC = "BestFit_11vo_SSA_v"+np.str(velocity)+"_M"+np.str(Mdot2)+"e-5.csv"
#tp = [100,200,500,800,1000,1200,1500,1800,2000,2200,2500,2800,2905,3000,3200,4000,5000,6000,7000,9000]
#tp = [100,200,500,800,1000,1200,1500,1800,2000,2200,2500,2800,3000,3200,3500,4000,5000,6000,7000,9000]
tp = [200,500,800,1000,1200,1500,1800,2000,2200,2500,2800,3000,3200]
#tp = [100,200,500,800,1000,1200,1500,1800,2000,2200,2500,2800,3000,3067,3200,3255,3500]

Fp_best = []
nu_p_best = []
R_best = []
vsh_best = []
M_best = []
    
for t in tp:
    #filein='SSAgrid_11aop_d'+np.str(t)+'.csv'
    filein='SSAgrid_11vo_d'+np.str(t)+'.csv'
    data=ascii.read(filein)
    
    #Figure out what radius it will be at this time:
    radius = velocity*1e5*t*86400.
    
    #Somehow interpolate the grid to our best fit? Prioritize Mdot? Then radius?
    #(1) Find all Mdot within a certain tolerance
    toleranceM = Mdot *0.01 #allow a 1% difference.
    maxiM = Mdot+toleranceM
    miniM = Mdot-toleranceM

    index = np.where((data['Mdot'] < maxiM))[0]
    data2=data[index]
    index2 = np.where(data2['Mdot'] > miniM)[0]
    data3=data2[index2]

    toleranceR = radius*0.01
    maxiR = radius+toleranceR
    miniR = radius-toleranceR
    index3 = np.where(data3['Radius'] < maxiR)[0]
    data4=data3[index3]
    index4 = np.where(data4['Radius'] > miniR)[0]
    data5=data4[index4]
    print('time',t,'numbers',len(index),len(index2),len(index3),len(index4))
    
    #Run a mini-grid in the region probed by the within 1% case. 
    
    F_p_2 = np.linspace(np.min(data5['F_peak']),np.max(data5['F_peak']),num=50) 
    nu_p_2 = np.linspace(np.min(data5['nu_peak']),np.max(data5['nu_peak']),num=50) 
    
    Fp_g2 = [] #peak flux
    nup_g2 =[] #peak frequency
    R_g2 =[] #radius
    vsh_g2 = [] #velocity of shock
    M_g2 = [] #mass loss rate
    
    for F in F_p_2:
        for nu in nu_p_2:
        
            #calculate Mass loss rate, Radius, velocity, etc.
            R,B,E,v,M = SSA_props(t,nu,F,D)
        
            # Append the values from this loop into the arrays that we defined above:
            Fp_g2.append(F) 
            nup_g2.append(nu) 
            R_g2.append(R) 
            vsh_g2.append(v) 
            M_g2.append(M)
    
    #Somehow figure out which in this grid is closest. 
    frac_diff_R = np.abs((radius - np.array(R_g2))/radius)
    frac_diff_M = np.abs((Mdot - np.array(M_g2))/Mdot)
    frac_diff = np.sqrt(frac_diff_R**2+frac_diff_M**2)
    
    indexN = np.where(frac_diff == np.min(frac_diff))[0]
    print('time',t,'differentials',frac_diff_R[indexN],frac_diff_M[indexN],frac_diff[indexN])
    
    #F_temp = 
    
    
    Fp_best.append(np.array(Fp_g2)[indexN][0])
    nu_p_best.append(np.array(nup_g2)[indexN][0])
    R_best.append(np.array(R_g2)[indexN][0])
    vsh_best.append(np.array(vsh_g2)[indexN][0])
    M_best.append(np.array(M_g2)[indexN][0])


tp=np.array(tp)
Fp_best = np.array(Fp_best)
nu_p_best = np.array(nu_p_best)
R_best = np.array(R_best)
vsh_best = np.array(vsh_best)
M_best = np.array(M_best)
# Write out the results into a data file that you can use later:
data = [tp,Fp_best,nu_p_best,R_best,vsh_best,M_best]
names = ['Time','F_peak','nu_peak','Radius','v_shock','Mdot']
ascii.write(data,fileout_LC,names=names,overwrite=True,format='csv')
